In [1]:
# Pacotes
# Bibliotecas de Modelagem de Dados e Análises
import pandas as pd
import numpy as np
import pandas_datareader.data as web


# Pacote para requisições HTTP
import requests
import os
from urllib.parse import urljoin

# Pacote para extrair arquivos.zip
from zipfile import ZipFile

# Manipular caminhos de arquivo e diretório de forma mais eficaz
from pathlib import Path

# Pacote para leitura e escrita de arquivos csv
import csv
import glob

# Pacote para tratar arquivos parquet
import pyarrow.parquet as pq
import pyarrow as pa

# Pacote para trabalhar com dados HTML e XML
from bs4 import BeautifulSoup

# Data
from datetime import datetime

# SQL
import sqlite3

# Avisos
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# URL das páginas que contém os links para os arquivos.zip das despesas  
url_despesas = 'https://dados.es.gov.br/dataset/despesas-municipios'

In [ ]:
# Pasta onde você deseja salvar os arquivos.zip
output_folder_despesas = 'C:/Users/Darke/Documents/Especialização - Dados/Analitycs e BI/TCC/arquivos/despesas'

# Certifique-se de que as pastas de saída existam
if not os.path.exists(output_folder_despesas):
    os.makedirs(output_folder_despesas)

In [ ]:
# Enviar uma solicitação HTTP para obter o conteúdo da página de despesas
response = requests.get(url_despesas)
if response.status_code == 200:
    # Analisar o HTML da página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar todos os links na página
    links = soup.find_all('a')

    # Iterar sobre os links da página de despesas
    for link in links:
        href = link.get('href')
        if href and href.endswith('.zip'):
            file_url = urljoin(url_despesas, href)
            file_name = os.path.join(output_folder_despesas, os.path.basename(file_url))

            # Baixar os arquivos de despesas.ZIP
            with requests.get(file_url, stream=True) as file_response:
                if file_response.status_code == 200:
                    with open(file_name, 'wb') as file:
                        for chunk in file_response.iter_content():
                            file.write(chunk)

                    print(f'Arquivo baixado: {file_name}')
                else:
                    print(f'Falha ao baixar {file_url}')
else:
    print(f'Falha ao acessar a página: {url_despesas}')

In [ ]:
# Especifica o caminho do diretório onde os arquivos ZIP das despesas estão localizados
caminho_arquivos_despesas = r'C:\Users\Darke\Documents\Especialização - Dados\Analitycs e BI\TCC\arquivos\despesas'

# Itera sobre todos os arquivos no diretório
for nome_arquivo_zip in os.listdir(caminho_arquivos_despesas):
    # Constrói o caminho completo para o arquivo ZIP
    caminho_zip = os.path.join(caminho_arquivos_despesas, nome_arquivo_zip)

    # Verifica se o arquivo tem a extensão .zip e é um arquivo válido
    if nome_arquivo_zip.endswith('.zip') and os.path.isfile(caminho_zip):
        # Abre o arquivo ZIP e extrai seu conteúdo no mesmo diretório
        with ZipFile(caminho_zip, 'r') as z:
            z.extractall(path=caminho_arquivos_despesas)
        
        # Exibe uma mensagem indicando que os arquivos foram extraídos com sucesso
        print(f'Arquivos extraídos em: {caminho_arquivos_despesas}')
    else:
        # Exibe uma mensagem se o arquivo ZIP não for válido ou não existir

        print(f'O arquivo ZIP "{nome_arquivo_zip}" não é válido ou não existe no caminho especificado.')

In [ ]:
# Alterando o enconding dos arquivos de ANSI para utf-8
def encode_files(path_despesas):
    # Informando o caminho do diretório
    folder = Path(path_despesas)

    # Iterando sobre os arquivos csv
    for path_file in folder.glob("*.csv"):
        output = folder / (path_file.stem + ".tmp")
        
        # Abre o arquivo CSV usando UTF-8. Em caso de erro, tenta novamente usando Latin-1
        try:
            with open(path_file, 'r', encoding="utf-8") as file:
                data = file.read()
        except UnicodeDecodeError:
            with open(path_file, 'r', encoding="latin-1") as file:
                data = file.read()
                
        # Abre o arquivo de saída temp e escreve os dados lidos do csv original
        with open(output, 'w', encoding="utf-8") as output_file:
            output_file.write(data)

        # Remove o csv original e renomeia o arquivo temporario
        path_file.unlink()
        output.rename(path_file)

if __name__ == "__main__":
    path_despesas = r'C:\Users\Darke\Documents\Especialização - Dados\Analitycs e BI\TCC\arquivos\despesas'
    encode_files(path_despesas)

Neste ponto foi necessária a correção manual em determinadas linhas dos arquivos de despesas de 2014 a 2017, pois havia dados na coluna "NomeDetalhamento" com ";".

Devido ao tamanho do arquivo de despesas, foi necessária a ação de conversão do csv para parquet antes da unificação.

In [ ]:
# Definir o diretório de trabalho
os.chdir(r'C:\Users\Darke\Documents\Especialização - Dados\Analitycs e BI\TCC\arquivos\despesas')

# Obter a lista de arquivos CSV no diretório
csv_files = glob.glob('*.csv')

# Loop através de cada arquivo CSV
for csv_file_path in csv_files:
    # Carregar o arquivo CSV para o DataFrame do pandas
    df = pd.read_csv(csv_file_path, sep=';', encoding='utf-8')

    # Construir o caminho para salvar o arquivo Parquet
    parquet_file_path = csv_file_path.replace('.csv', '.parquet')

    # Salvar o DataFrame como um arquivo Parquet
    df.to_parquet(parquet_file_path, index=False)

    # Mensagens de debug
    print(f'O arquivo CSV {csv_file_path} foi convertido com sucesso para Parquet em: {parquet_file_path}')


In [ ]:
# Correção necessária em dois arquivos 2022 e 2023
caminho_arquivo_parquet_2023 = r'C:\Users\Darke\Documents\Especialização - Dados\Analitycs e BI\TCC\arquivos\despesas\municipios-despesas-2023.parquet'
caminho_arquivo_parquet_2022 = r'C:\Users\Darke\Documents\Especialização - Dados\Analitycs e BI\TCC\arquivos\despesas\municipios-despesas-2022.parquet'
caminho_arquivo_parquet_2021 = r'C:\Users\Darke\Documents\Especialização - Dados\Analitycs e BI\TCC\arquivos\despesas\municipios-despesas-2021.parquet'

# Carregando o arquivo Parquet em um DataFrame
df1 = pd.read_parquet(caminho_arquivo_parquet_2023)
df2 = pd.read_parquet(caminho_arquivo_parquet_2022)
df3 = pd.read_parquet(caminho_arquivo_parquet_2021)

# Convertendo as colunas para int64 e float64
colunas_para_converter_1 = ["CodigoOrgao", "CodigoUnidadeOrcamentaria"]
colunas_para_converter_2 = ["CodigoAcao"]
df1[colunas_para_converter_1] = df1[colunas_para_converter_1].astype("int64")
df2[colunas_para_converter_1] = df2[colunas_para_converter_1].astype("int64")
df3[colunas_para_converter_1] = df3[colunas_para_converter_1].astype("int64")
df1[colunas_para_converter_2] = df1[colunas_para_converter_2].astype("float64")
df2[colunas_para_converter_2] = df2[colunas_para_converter_2].astype("float64")
df3[colunas_para_converter_2] = df3[colunas_para_converter_2].astype("float64")

# Salvando o DataFrame de volta no formato Parquet
df1.to_parquet(caminho_arquivo_parquet_2023, index=False)
df2.to_parquet(caminho_arquivo_parquet_2022, index=False)
df3.to_parquet(caminho_arquivo_parquet_2021, index=False)

In [2]:
# Unificando os arquivos .parquet
def merge_parquet_files(path_despesas):
    folder = Path(path_despesas)
    output_file_path = folder / "despesas_unificadas.parquet"
    merged_table = None

    # Iterando sobre cada arquivo Parquet
    for path_file in folder.glob("*.parquet"):
        # Leitura do arquivo Parquet
        table = pq.read_table(str(path_file))

        # Se for o primeiro arquivo, crie a tabela mesclada
        if merged_table is None:
            merged_table = table
        else:
            # Mesclar a tabela atual com a tabela mesclada
            merged_table = pa.concat_tables([merged_table, table])

    # Escrever a tabela mesclada no arquivo de saída
    pq.write_table(merged_table, str(output_file_path))

if __name__ == "__main__":
    path_despesas = r'C:\Users\Darke\Documents\Especialização - Dados\Analitycs e BI\TCC\arquivos\despesas'
    merge_parquet_files(path_despesas)

In [2]:
arquivo_despesa = r'C:\Users\Darke\Documents\Especialização - Dados\Analitycs e BI\TCC\arquivos\despesas\despesas_unificadas.parquet'
dados = pd.read_parquet(arquivo_despesa)

In [3]:
# Validando os anos no arquivo de despesas unificado
dados['Ano'] = pd.to_numeric(dados['Ano'], errors='coerce')
print(dados['Ano'].unique())

[2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023]


In [4]:
# Cria a nova coluna data com lambda e datetime
dados["data"] = dados.apply(lambda row: datetime(int(row["Ano"]), int(row["Mes"]), 1), axis=1)


In [5]:
dados.head()

,Ano,Mes,CodigoUnidadeGestora,UnidadeGestora,EsferaAdministrativa,CodigoOrgao,NomeOrgao,CodigoUnidadeOrcamentaria,NomeUnidadeOrcamentaria,CodigoFuncao,...,CodigoFonteReduzida,NomeFonteReduzida,CodigoDetalhamento,NomeDetalhamento,PrevisaoInicial,PrevisaoAtualizada,Empenhada,Liquidada,Paga,data
0,2013,6,001E0700001,Prefeitura Municipal de Afonso Cláudio,Afonso Cláudio,7,SECRETARIA MUNICIPAL DE ADMINISTRACAO,1,SECRETARIA MUNICIPAL DE ADMINISTRACAO,4,...,0,RECURSOS ORDINÁRIOS,0,RECURSOS ORDINÁRIOS,"0,00","0,00","0,00","49798,73","49798,73",2013-06-01
1,2013,7,001E0700001,Prefeitura Municipal de Afonso Cláudio,Afonso Cláudio,2,GABINETE DO PREFEITO,1,GABINETE DO PREFEITO,4,...,0,RECURSOS ORDINÁRIOS,0,RECURSOS ORDINÁRIOS,"0,00","0,00","0,00","2227,80","2227,80",2013-07-01
2,2013,9,001E0700001,Prefeitura Municipal de Afonso Cláudio,Afonso Cláudio,2,GABINETE DO PREFEITO,1,GABINETE DO PREFEITO,4,...,0,RECURSOS ORDINÁRIOS,0,RECURSOS ORDINÁRIOS,"0,00","0,00","0,00","3034,00","807,00",2013-09-01
3,2013,7,001E0700001,Prefeitura Municipal de Afonso Cláudio,Afonso Cláudio,14,SECRETARIA MUNICIPAL DE ESPORTE E LAZER,3,SECRETARIA MUNICIPAL DE ESPORTE E LAZER,4,...,0,RECURSOS ORDINÁRIOS,0,RECURSOS ORDINÁRIOS,"0,00","0,00","0,00","1320,00","1320,00",2013-07-01
4,2013,9,001E0700001,Prefeitura Municipal de Afonso Cláudio,Afonso Cláudio,6,SECRETARIA MUNICIPAL DE AÇÃO SOCIAL,1,FUNDO MUNICIPAL DE ASSISTÊNCIA SOCIAL,8,...,0,RECURSOS ORDINÁRIOS,0,RECURSOS ORDINÁRIOS,"0,00","0,00","0,00","5249,56","104,99",2013-09-01


In [3]:
# Somar o total de valores nulos em cada coluna
total_nulos_por_coluna = dados.isnull().sum()

# Somar o total de valores nulos em todo o DataFrame
total_nulos_no_dataframe = dados.isnull().sum().sum()

print("Total de nulos por coluna:")
print(total_nulos_por_coluna)

print("\nTotal de nulos no DataFrame:")
print(total_nulos_no_dataframe)

Total de nulos por coluna:
ano                               0
mes                               0
codigo_unidade_gestora            0
unidade_gestora                   0
esfera_administrativa             0
codigo_orgao                      0
nome_orgao                     8833
codigo_unidade_orcamentaria       0
nome_unidade_orcamentaria      8414
codigo_funcao                     0
descricao_funcao                  0
codigo_sub_funcao                 0
descricao_sub_funcao              0
codigo_programa                   0
nome_programa                    96
codigo_acao                       0
nome_acao                         0
codigo_completo                   0
codigo_categoria                  0
descricao_categoria               0
codigo_grupo_natureza             0
DescricaoGrupoNatureza            0
codigo_modalidade                 0
DescricaoModalidade               0
codigo_elemento                   0
descricao_elemento                0
codigo_sub_elemento               0
d

In [4]:
# Renomeando as colunas
novos_nomes = {
    'Ano': 'ano',
    'Mes': 'mes',
    'CodigoUnidadeGestora': 'codigo_unidade_gestora',
    'UnidadeGestora': 'unidade_gestora',
    'EsferaAdministrativa': 'esfera_administrativa',
    'CodigoOrgao': 'codigo_orgao',
    'NomeOrgao': 'nome_orgao',
    'CodigoUnidadeOrcamentaria': 'codigo_unidade_orcamentaria',
    'NomeUnidadeOrcamentaria': 'nome_unidade_orcamentaria',
    'CodigoFuncao': 'codigo_funcao',
    'DescricaoFuncao': 'descricao_funcao',
    'CodigoSubFuncao': 'codigo_sub_funcao',
    'DescricaoSubFuncao': 'descricao_sub_funcao',
    'CodigoPrograma': 'codigo_programa',
    'NomePrograma': 'nome_programa',
    'CodigoAcao': 'codigo_acao',
    'NomeAcao': 'nome_acao',
    'CodigoCompleto': 'codigo_completo',
    'CodigoCategoria': 'codigo_categoria',
    'DescricaoCategoria': 'descricao_categoria',
    'CodigoGrupoNatureza': 'codigo_grupo_natureza',
    'DescricaoGrupoNatureza': 'descricao_grupo_natureza',
    'CodigoModalidade': 'codigo_modalidade',
    'DescricaoModalidade': 'descricao_modalidade',
    'CodigoElemento': 'codigo_elemento',
    'DescricaoElemento': 'descricao_elemento',
    'CodigoSubElemento': 'codigo_sub_elemento',
    'DescricaoSubElemento': 'descricao_sub_elemento',
    'CodigoGrupoFonte': 'codigo_grupo_fonte',
    'NomeGrupoFonte': 'nome_grupo_fonte',
    'CodigoFonteReduzida': 'codigo_fonte_reduzida',
    'NomeFonteReduzida': 'nome_fonte_reduzida',
    'CodigoDetalhamento': 'codigo_detalhamento',
    'NomeDetalhamento': 'nome_detalhamento',
    'PrevisaoInicial': 'previsao_inicial',
    'PrevisaoAtualizada': 'previsao_atualizada',
    'Empenhada': 'empenhada',
    'Liquidada': 'liquidada',
    'Paga': 'paga',
    'data': 'data',
}

dados.rename(columns=novos_nomes, inplace=True)

In [8]:
# Incluindo novo campo de data_carga no arquivo
data_carga = datetime.now()
dados['data_carga'] = data_carga

In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8352473 entries, 0 to 8352472
Data columns (total 41 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   ano                          int64         
 1   mes                          int64         
 2   codigo_unidade_gestora       object        
 3   unidade_gestora              object        
 4   esfera_administrativa        object        
 5   codigo_orgao                 int64         
 6   nome_orgao                   object        
 7   codigo_unidade_orcamentaria  int64         
 8   nome_unidade_orcamentaria    object        
 9   codigo_funcao                int64         
 10  descricao_funcao             object        
 11  codigo_sub_funcao            int64         
 12  descricao_sub_funcao         object        
 13  codigo_programa              int64         
 14  nome_programa                object        
 15  codigo_acao                  float64       
 16  

In [6]:
# Salva as alterações de volta ao arquivo original
dados.to_parquet(arquivo_despesa, index=False)